# Create Nightly statistics of frames



In [1]:
import os, glob
import numpy as np
from astropy.io import fits, ascii
import datetime
from astropy.table import Table, unique
from astropy.time import Time

# For matplotlib plotting
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator)
# import request to query archive
import requests

matplotlib.rcParams["figure.figsize"] = (8, 6)
matplotlib.rcParams["font.size"] = 10
matplotlib.rcParams["figure.dpi"] = 120

#define the initial basename for the NRES files in archive:
basename = {   'cpt' : 'cptnrs03',
                'elp' : 'elpnrs02',
                'lsc' : 'lscnrs01',
                'tlv' : 'tlvnrs04',
            }

#define date string format for query api
fmt_api = '%Y-%m-%d'
dt1 = datetime.timedelta(days=1)

## Prepare notebook to download data from the archive

You need a `userdat.dat` file in your home directory or somewhere else. This file has the following two lines:
```
username=
password=
```
You have to insert user name and password after `=` sign.



In [2]:
# Get data from user file:
user_file = open('/home/mrabus/userdata.dat', 'r') # File with the username and password to access the archive
#read username and password
username  = (user_file.readline().split('=')[-1]).split()[0]
password  = (user_file.readline().split('=')[-1]).split()[0]
# Request token and create header to access the archive
api_token = requests.post('https://archive-api.lco.global/api-token-auth/',data = {'username': username,'password': password}).json() 
headers={'Authorization': 'Token '+api_token['token']}  

In [3]:
#site we want to check
site = 'cpt' 
# starting night we want to check as datetime object
start_night = datetime.datetime(2019,5,20)
# number of nights we want to check
nr_nights = 5

nightly_date   = []
nightly_bias   = []
nightly_dark   = []

nightly_flat   = {}
nightly_flat[0] = []
nightly_flat[1] = []
nightly_arc    = {}
nightly_arc[0] = []
nightly_arc[1] = []
nightly_target = {}
nightly_target[0] = []
nightly_target[1] = []


for ii_night in range(nr_nights):
    
    night_offset = datetime.timedelta(days=ii_night)
    
    # create start and end of night in UTC, according to site.
    if site == 'cpt':
        sdate = (start_night + night_offset).strftime(fmt_api)     +' 11%3A30'
        edate = (start_night + night_offset + dt1).strftime(fmt_api)+' 11%3A00'
    if site == 'elp':
        sdate = (start_night + night_offset).strftime(fmt_api)     +' 18%3A30'
        edate = (start_night + night_offset + dt1).strftime(fmt_api)+' 18%3A00'
    if site == 'lsc':
        sdate = (start_night + night_offset).strftime(fmt_api)     +' 16%3A30'
        edate = (start_night + night_offset + dt1).strftime(fmt_api)+' 16%3A00'
    if site == 'tlv':
        sdate = (start_night + night_offset).strftime(fmt_api)     +' 10%3A30'
        edate = (start_night + night_offset + dt1).strftime(fmt_api)+' 10%3A00'

    # Query bias frames:
    response_bias = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                                 edate+'&OBSTYPE=BIAS&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                                 headers=headers).json() 
    # Query dark frames:
    response_dark = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                                 edate+'&OBSTYPE=DARK&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                                 headers=headers).json() 
    # Query lamp flats:
    response_flat = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                                 edate+'&OBSTYPE=LAMPFLAT&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                                 headers=headers).json() 
    # Query arcs:
    response_arc = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                                 edate+'&OBSTYPE=DOUBLE&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                                 headers=headers).json() 
    # Query target/science expousres:
    response_target = requests.get('https://archive-api.lco.global/frames/?start='+sdate+'&end='+ \
                                edate+'&OBSTYPE=TARGET&basename='+basename[site]+'&RLEVEL=0&TELID=igla', \
                                headers=headers).json()
    
    nightly_date.append( (start_night + night_offset).strftime(fmt_api) )
    
    
    
    
    ####################
    #
    # Count targets
    #
    ####################
    
    target0 = 0
    target1 = 0
    
    frames = response_target['results']
    for frame in frames:
        frame_id = frame['id']
        #print(frame['basename'])
        hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                                headers=headers).json()
        
        if  hdr['data']['SHUTTER0'] == 'open':
            target0 += 1
        elif  hdr['data']['SHUTTER1'] == 'open':
            target1 += 1
    
    nightly_target[0].append(target0)
    nightly_target[1].append(target1)
    


    ####################
    #
    # Count flats
    #
    ####################    
    
    flat0 = 0
    flat1 = 0
    
    frames = response_flat['results']
    for frame in frames:
        frame_id = frame['id']
        #print(frame['basename'])
        hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                                headers=headers).json()
        
        if  hdr['data']['SHUTTER0'] == 'open':
            flat0 += 1
        elif  hdr['data']['SHUTTER1'] == 'open':
            flat1 += 1
    
    nightly_flat[0].append(flat0)
    nightly_flat[1].append(flat1)

    
    
    
    ####################
    #
    # Count arcs
    #
    ####################    
    
    arc0 = 0
    arc1 = 0
    
    frames = response_arc['results']
    for frame in frames:
        frame_id = frame['id']
        #print(frame['basename'])
        hdr = requests.get(f'https://archive-api.lco.global/frames/{frame_id}/headers/', \
                                headers=headers).json()
        
        if  hdr['data']['SHUTTER0'] == 'open':
            arc0 += 1
        elif  hdr['data']['SHUTTER1'] == 'open':
            arc1 += 1
    
    nightly_arc[0].append(arc0)
    nightly_arc[1].append(arc1)

    
    print( f'Night {(start_night + night_offset).strftime(fmt_api)}' )
    

Night 2019-05-20
Night 2019-05-21
Night 2019-05-22
Night 2019-05-23
Night 2019-05-24


In [4]:
fig, ax = plt.subplots()
plt.plot_date(nightly_date, nightly_flat[0], fmt='o', ls='-', color='blue', ms=12, 
              label='flat SHUTTER0 open')
plt.plot_date(nightly_date, nightly_arc[0], fmt='o', ls='-', color='red', ms=8, 
              label='arc SHUTTER0 open')
plt.plot_date(nightly_date, nightly_target[0], fmt='o', ls='-', color='green', ms=4, 
              label='target SHUTTER0 open')
plt.plot_date(nightly_date, nightly_flat[1], fmt='o', ls=':', color='blue', ms=12, 
              label='flat SHUTTER1 open')
plt.plot_date(nightly_date, nightly_arc[1], fmt='o', ls=':', color='red', ms=8, 
              label='arc SHUTTER1 open')
plt.plot_date(nightly_date, nightly_target[1], fmt='o', ls=':', color='green', ms=4, 
              label='target SHUTTER1 open')
ax.xaxis.set_tick_params(rotation=30)
ax.set_ylabel('Number of frames')
ax.set_xlabel('Night')

plt.legend()

FigureCanvasNbAgg()